In [133]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [134]:
from IPython.display import display, HTML
display(HTML("<style>.output_wrapper, .output {height:auto !important; max-height: 300px; overflow-y: auto;}</style>"))


In [135]:
pip install langgraph langchain google-generativeai


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [136]:
pip install langchain faiss-cpu sentence-transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [137]:
pip install -U langchain-community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [138]:
pip install pypdf


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [139]:
from langgraph.graph import StateGraph, END


In [140]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/Users/RuchithaPadala/Downloads/Langchain concepts.pdf")
documents = loader.load()



In [141]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings()
db = FAISS.from_documents(documents, embedding)


/var/folders/c3/lnf3w9n16s19gmr0gqzztl480000gr/T/ipykernel_12381/2074367401.py:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings()


In [142]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = splitter.split_documents(documents)

db = FAISS.from_documents(split_docs, embedding)


In [143]:
print(f"Total documents loaded: {len(documents)}")
print(documents[0].page_content[:1000])


Total documents loaded: 2
Title:  LangChain  Overview  for  Developers   LangChain  is  a  framework  designed  to  simplify  the  development  of  applications  powered  by  
large
 
language
 
models
 
(LLMs).
 
It
 
provides
 
modular
 
components
 
and
 
tools
 
to
 
build
 
custom
 
AI
 
agents,
 
chatbots,
 
question-answering
 
systems,
 
and
 
more.
  1.  Key  Concepts   -  Language  Models:  These  are  foundational  models  like  GPT-4  or  Gemini  that  generate  
human-like
 
text.
 -  Chains:  A  series  of  calls  that  link  together  components  like  prompts,  tools,  or  other  chains.  -  Agents:  These  make  decisions  about  which  tools  to  use  based  on  user  input.  -  Memory:  This  allows  AI  applications  to  remember  previous  interactions  for  more  personalized  
responses.
 -  Retrieval:  Integration  with  document  stores  to  allow  grounding  AI  responses  on  custom  or  
external
 
knowledge.
  2.  Use  Cases   -  Customer  Support  Bots  - 

In [144]:
def retrieve_internal_docs(query: str, top_k: int = 3, threshold: float = 0.75, keyword_ratio: float = 0.3) -> str:
    results = db.similarity_search_with_score(query, k=top_k)

    print("🔍 RAG similarity scores:")
    for doc, score in results:
        print(f"📏 Score: {score:.2f}")
    
    # Step 1: Filter by threshold
    candidates = [(doc, score) for doc, score in results if score <= threshold]

    # Step 2: Filter further by keyword overlap
    from collections import Counter

    query_words = set(query.lower().split())

    final_docs = []
    for doc, score in candidates:
        doc_words = set(doc.page_content.lower().split())
        overlap = query_words & doc_words
        ratio = len(overlap) / len(query_words) if query_words else 0
        print(f"🔠 Overlap ratio: {ratio:.2f}")

        if ratio >= keyword_ratio:
            final_docs.append(doc)

    if not final_docs:
        print("❌ RAG rejected — No meaningful document found.")
        return ""

    print("✅ RAG accepted — returning document(s).")
    return "\n\n".join([doc.page_content for doc in final_docs])


In [145]:
pip install google-search-results


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [146]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env

# Now fetch the key
SERP_API_KEY = os.getenv("SERPAPI_API_KEY")


True

In [147]:
def search_google_web(query: str, max_results: int = 5) -> str:
    from serpapi import GoogleSearch
    import re

    params = {
        "engine": "google",
        "q": query,
        "api_key": SERP_API_KEY,
        "num": max_results
    }
    search = GoogleSearch(params)
    results = search.get_dict()

    if not results.get("organic_results"):
        return "No reliable web result found."

    for res in results["organic_results"]:
        title = res.get("title", "")
        snippet = res.get("snippet", "")
        link = res.get("link", "")

        # 1. ❌ Reject links with wrong year
        if re.search(r"202[0-4]", snippet) or "2023" in link:
            continue

        # 2. ✅ Strong match: 2025 + winner keywords
        if "2025" in snippet and any(word in snippet.lower() for word in ["won", "defeated", "champion", "beat", "final"]):
            return f"{title}: {snippet} (Source: {link})"

        # 3. ✅ Reputable fallback with 2025
        if "2025" in snippet and any(domain in link for domain in ["wikipedia.org", "espncricinfo.com"]):
            return f"{title}: {snippet} (Source: {link})"

    # 4. Final fallback (less strict)
    res = results["organic_results"][0]
    return f"{res.get('title')}: {res.get('snippet')} (Source: {res.get('link')})"


In [148]:
# Basic Python tools
from dataclasses import dataclass
from typing import Annotated

# LangGraph: To build the AI agent graph
from langgraph.graph import StateGraph, END

# LangChain: To work with Google Gemini
from langchain_google_genai import ChatGoogleGenerativeAI

# For schema validation of input/output
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [149]:
from dotenv import load_dotenv
import os



# Load .env file
load_dotenv()

# Fetch the key from the environment
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Check that it's loaded
print("✅ API Key loaded!" if GOOGLE_API_KEY else "❌ API Key not found.")


True

✅ API Key loaded!


In [150]:
import google.generativeai as genai

llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash-002",
    convert_system_message_to_human=False,  # 🔁 Disable auto-conversion
    temperature=0.7
)





In [151]:
genai.configure(api_key=GOOGLE_API_KEY)

gemini_model = genai.GenerativeModel("gemini-1.5-flash")  # or gemini-1.5-pro


In [152]:
from dataclasses import dataclass, field, replace
from dataclasses import dataclass
from typing import Optional

@dataclass
class AgentState:
    query: str
    analysis: Optional[str] = None
    response: Optional[str] = None
    source: Optional[str] = None  # ✅ ADD THIS LINE


In [153]:
def input_node(state: AgentState) -> AgentState:
    print("📥 Input received:", state.query)
    return state

In [154]:
pip install langgraph langchain google-generativeai


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [155]:
pip install langchain faiss-cpu sentence-transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [156]:
pip install -U langchain-community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [157]:
pip install pypdf


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [158]:
def retrieve_internal_docs(query: str, top_k: int = 3, threshold: float = 0.75) -> str:
    results = db.similarity_search_with_score(query, k=top_k)
    
    for doc, score in results:
        print(f"📏 Score (L2 Distance): {score:.2f}")
    
    # ✅ Correct: keep only documents with LOW distance (more similar)
    filtered = [doc for doc, score in results if score <= threshold]

    if not filtered:
        print("❌ RAG rejected — all documents were above the distance threshold.")
        return ""

    print("✅ RAG accepted — returning result")
    return "\n\n".join([doc.page_content for doc in filtered])


In [159]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = splitter.split_documents(documents)

db = FAISS.from_documents(split_docs, embedding)


In [160]:
pip install google-search-results


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [161]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env

# Now fetch the key
SERP_API_KEY = os.getenv("SERPAPI_API_KEY")


True

In [162]:
def search_google_web(query: str, max_results: int = 5) -> str:
    from serpapi import GoogleSearch
    import re

    params = {
        "engine": "google",
        "q": query,
        "api_key": SERP_API_KEY,
        "num": max_results
    }
    search = GoogleSearch(params)
    results = search.get_dict()

    if not results.get("organic_results"):
        return "No reliable web result found."

    prime_minister_names = ["Keir Starmer", "Rishi Sunak"]  # Expand if needed

    # 1. Try to find a reliable snippet with a known PM name
    for res in results["organic_results"]:
        title = res.get("title", "")
        snippet = res.get("snippet", "")
        link = res.get("link", "")

        if any(name in snippet for name in prime_minister_names):
            return f"{snippet} (Source: {link})"

    # 2. As fallback, return the most relevant snippet even if generic
    top_result = results["organic_results"][0]
    return f"{top_result.get('title')}: {top_result.get('snippet')} (Source: {top_result.get('link')})"


In [163]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Fetch the key from the environment
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Check that it's loaded
print("✅ API Key loaded!" if GOOGLE_API_KEY else "❌ API Key not found.")


True

✅ API Key loaded!


In [164]:
import google.generativeai as genai

llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash-002",
    convert_system_message_to_human=False,  # 🔁 Disable auto-conversion
    temperature=0.7
)





In [165]:
genai.configure(api_key=GOOGLE_API_KEY)

gemini_model = genai.GenerativeModel("gemini-1.5-flash")  # or gemini-1.5-pro


In [166]:
from dataclasses import dataclass

@dataclass
class AgentState:
    query: str
    analysis: str = ""
    research: str = ""
    response: str = ""
    source: str = ""            # ✅ ADD THIS LINE
    validated: bool = False
    iteration: int = 0
    max_iterations: int = 3


In [167]:
initial_state = AgentState(
    query= "Tell me a philosophical thought about time?",
    analysis="",
    research="",
    response="",
    validated=False,
    iteration=0,
    max_iterations=3
)



In [168]:
def input_node(state: AgentState) -> AgentState:
    print("📥 Input received:", state.query)
    return state

In [169]:
def analyzer_node(state: AgentState) -> AgentState:
    print("📊 Analyzing query:", state.query)
    try:
        result = gemini_model.generate_content(f"Analyze this user question for intent:\n{state.query}")
        analysis = result.text.strip()
    except Exception as e:
        print("⚠️ Analyzer error:", e)
        analysis = "Could not analyze the query."
    print("🔍 Analysis:", analysis)
    return replace(state, analysis=analysis)


In [170]:
def researcher_node(state: AgentState) -> AgentState:
    query = state.query
    print(f"🔬 Researcher activated for query: {query}")

    rag_result = retrieve_internal_docs(query, threshold=0.75)

    if rag_result.strip():
        print("📚 Using RAG")
        return replace(state, research=rag_result, source="RAG")

    web_result = search_google_web(query)
    if web_result and "No reliable" not in web_result:
        print("🌐 Using Web")
        return replace(state, research=web_result, source="WEB")

    llm_result = call_gemini_llm(query)
    print("🤖 Using Gemini LLM")
    return replace(state, research=llm_result, source="LLM")


In [171]:
def call_gemini_llm(query: str) -> str:
    try:
        result = gemini_model.generate_content(query)
        return result.text.strip()
    except Exception as e:
        print("⚠️ LLM Error:", e)
        return "LLM could not generate a response."


In [172]:
def responder_node(state: AgentState) -> AgentState:
    print("🧠 Generating response for:", state.query)
    try:
        if "no confirmed answer" in state.research.lower():
            response = state.research
        else:
            source_icon = {
                "RAG": "📚",
                "WEB": "🌐",
                "LLM": "🤖"
            }.get(state.source, "❓")

            response = f"{source_icon} Answer based on {state.source}:\n\n{state.research}"
    except Exception as e:
        print("⚠️ Responder error:", e)
        response = "Response generation failed."

    print("💬 Response (first 500 chars):", response[:500], "...")
    return replace(state, response=response)


In [173]:
def validator_node(state: AgentState) -> AgentState:
    print("✅ Validating response...")
    query = state.query
    answer = state.research

    try:
        internal_info = retrieve_internal_docs(query)
        if internal_info and "no matching" not in internal_info.lower():
            print("📚 Using RAG for validation.")
            context = f"📚 Internal Docs:\n{internal_info}"
        else:
            web_info = search_google_web(query)
            if web_info and "no reliable" not in web_info.lower():
                print("🌐 RAG failed. Using Web for validation.")
                context = f"🌐 Web Info:\n{web_info}"
            else:
                print("🤖 RAG & Web failed. Using Gemini fallback for validation.")
                prompt = f"""
Validate the answer below for the question. If it's factual, return ✅. If wrong or unverified, return ❌.

Question: {query}
Answer: {answer}

Respond with ✅ or ❌ only.
"""
                result = gemini_model.generate_content(prompt)
                result_text = result.text.strip()
                print("🧪 Gemini validation reply:", result_text)
                validated = result_text.startswith("✅")
                return replace(state, validated=validated)

        prompt = f"""
Validate this answer using only the content below. Do NOT assume or hallucinate.

Question: {query}
Answer: {answer}

{context}

Instructions:
- If the answer is fully supported, reply ✅
- If the answer is incorrect or missing context, reply ❌
Reply with only ✅ or ❌.
"""
        result = gemini_model.generate_content(prompt)
        result_text = result.text.strip()
        print("🧪 Gemini validation reply:", result_text)
        validated = result_text.startswith("✅")
        return replace(state, validated=validated)

    except Exception as e:
        print("⚠️ Validator Node Error:", str(e))
        return replace(state, validated=False)


In [174]:
def router_node(state: AgentState) -> dict:
    if state.validated or state.iteration >= state.max_iterations:
        return {"end": END}
    return {"analyzer": "analyzer"}


In [175]:
workflow = StateGraph(AgentState)

# Add core nodes
workflow.add_node("input", lambda state: state)
workflow.add_node("analyzer", analyzer_node)
workflow.add_node("researcher", researcher_node)
workflow.add_node("validator", validator_node)
workflow.add_node("responder", responder_node)

# Set entry point
workflow.set_entry_point("input")

# Add edges
workflow.add_edge("input", "analyzer")
workflow.add_edge("analyzer", "researcher")
workflow.add_edge("researcher", "validator")
workflow.add_edge("validator", "responder")

# Finish
workflow.set_finish_point("responder")

# Compile
app = workflow.compile()


In [176]:
# 🚀 Run the workflow
import re


raw_final_state = app.invoke(initial_state)
final_state = AgentState(**raw_final_state)

# 🧾 Format response
cleaned_response = re.sub(r'\s*\n\s*', ' ', final_state.response).strip()

# Print to console
print("\n🧾 Final Response:", cleaned_response)
print("🔁 Iterations:", final_state.iteration)
print("✅ Validated:", final_state.validated)

# 🖥️ HTML display
display(HTML(f"""
<div style="border:1px solid #ccc; padding:10px; max-height:300px; overflow:auto; white-space:pre-wrap;">
<b>🧾 Final Response:</b><br>{cleaned_response}<br><br>
<b>🔁 Iterations:</b> {final_state.iteration}<br>
<b>✅ Validated:</b> {final_state.validated}
</div>
"""))


📊 Analyzing query: Tell me a philosophical thought about time?
🔍 Analysis: The user's intent is to **receive a philosophical reflection or contemplation on the nature of time.**  They are seeking insight into a philosophical concept, not a factual explanation.  They want to explore a thought-provoking idea related to time, potentially exploring its subjective nature, its linearity or non-linearity, its relationship to existence, or other metaphysical aspects.
🔬 Researcher activated for query: Tell me a philosophical thought about time?
📏 Score (L2 Distance): 1.71
📏 Score (L2 Distance): 1.85
📏 Score (L2 Distance): 1.88
❌ RAG rejected — all documents were above the distance threshold.
🌐 Using Web
✅ Validating response...
📏 Score (L2 Distance): 1.71
📏 Score (L2 Distance): 1.85
📏 Score (L2 Distance): 1.88
❌ RAG rejected — all documents were above the distance threshold.
🌐 RAG failed. Using Web for validation.
🧪 Gemini validation reply: ✅
🧠 Generating response for: Tell me a philosophical t